In [ ]:
library(tidyverse)
library(MASS)
library(glmnet)

source('./simulation_utils.R')

options(repr.plot.width=6, repr.plot.height=4) # plot sizes in this notebook


In [ ]:
# set.seed(089342)
set.seed(453452)

# Fixed parameters in our analysis

In [ ]:
d <- 20 # total number of dimensions
n_cs <- 1000 # number of observations in construction sample
n_vs <- 10000 # number of observations in validation sample

In [ ]:
# draw X for construction sample. This will be fixed for our analysis 
covar_x <- diag(d) * 10. # true covariance of our draws of X
mu_x <- rep(0, d) # mean of our draws of X

x_cs_ <- mvrnorm(n_cs, mu_x, covar_x) 
x_cs <- scale(x_cs_, scale = FALSE, center = TRUE) # center the columns of X

# design covariance: AR(1) structure
# rho <- 0.9
# x_cs <- rho**abs(outer(1:d, 1:d, "-"))

In [ ]:
# regression coefficients
beta_full <- rnorm(d, 0, 1)

In [ ]:
sigma <- 50 # standard error of noise

s <- 0 # sparsity; for ridge, we do not assume sparsity

In [ ]:
xbeta <- x_cs %*% beta_full

In [ ]:
1 / 1000 * t(xbeta) %*% xbeta 

In [ ]:
50 **2

# One round of analysis

## Draw data: 

In [ ]:
data <- draw_cs_vs_sample(x_cs, beta_full, s, sigma, n_vs)

In [ ]:
plot(data$x_cs %*% data$true_beta, data$y_cs)

## run ridge regression

In [ ]:
ridge_fit <- glmnet(data$x_cs, data$y_cs, alpha = 0.0)
lambdas_path <- ridge_fit$lambda # the lambdas the path examined

In [ ]:
lambdas <- c(lambdas_path)
n_lambdas <- length(lambdas)

In [ ]:
plot(ridge_fit)

## Select lambda using cross-validation

In [ ]:
lambda_cv <- choose_lambda_cv(data$x_cs, data$y_cs, lambdas, alpha = 0, plot = TRUE)

In [ ]:
predicted_cs_y <- predict(ridge_fit, x = data$x_cs, y = data$y_cs, newx = data$x_cs, s = lambdas, exact = TRUE)
predicted_vs_y <- predict(ridge_fit, x = data$x_cs, y = data$y_cs, newx = data$x_vs, s = lambdas, exact = TRUE)

In [ ]:
cs_slopes <- get_slope(predicted_cs_y, data$y_cs)
vs_slopes <- get_slope(predicted_vs_y, data$y_vs)

data.frame(slopes = c(as.vector(cs_slopes), as.vector(vs_slopes)), 
           lambda = c(lambdas, lambdas), 
           sample = c(rep('train', n_lambdas), rep('test', n_lambdas))) %>%
    filter(is.finite(slopes)) %>%
    ggplot() + geom_point(aes(x = log(lambda), y = log(slopes), color = sample), size = 1) + 
        geom_hline(yintercept = 0) + 
        geom_vline(xintercept = log(lambda_cv), linetype = 'dashed', color = 'red') + 
        scale_color_manual(values=c("royalblue1", "gold")) + theme_bw() + 
    theme(text = element_text(size = 25), plot.title = element_text(hjust = 0.5))


ggsave('../writing/ridge_figures/slope_vs_lambda.png', width = 9, height = 6)

In [ ]:
# slope is observed y on predicted y

In [ ]:
# for small lambda we get the regression case again
# should should be less than 1 a la Copas
vs_slopes[which.min(lambdas)]

In [ ]:
# should be close to 1
# we should be able to fit the line perfectly with enough coefficients
cs_slopes[which.min(lambdas)]

In [ ]:
cs_mspe <- get_mspe(predicted_cs_y, data$y_cs)
vs_mspe <- get_mspe(predicted_vs_y, data$y_vs)

data.frame(mspe = c(as.vector(cs_mspe), as.vector(vs_mspe)), 
           lambda = c(lambdas, lambdas), 
           sample = c(rep('construction', n_lambdas), rep('validation', n_lambdas))) %>%
    filter(is.finite(mspe)) %>%
    ggplot() + geom_point(aes(x = log(lambda), y = mspe, color = sample)) 

In [ ]:
log(lambdas)[which.min(vs_mspe)]

In [ ]:
predicted_vs_y_lambda_min <- 
    predict(ridge_fit, x = data$x_cs, y = data$y_cs, newx = data$x_vs, s = lambda_cv, exact = TRUE)
predicted_vs_y_ols <- 
    predict(ridge_fit, x = data$x_cs, y = data$y_cs, newx = data$x_vs, s = 0, exact = TRUE)

data.frame(y_obs = data$y_vs, 
           ridge = as.vector(predicted_vs_y_lambda_min), 
            ols = as.vector(predicted_vs_y_ols)) %>%
    mutate(ridge = ridge - mean(ridge)) %>%
    mutate(ols = ols - mean(ols)) %>% 
    mutate(y_obs = y_obs - mean(y_obs)) %>%
    gather(key = 'procedure', value = 'y_pred', ridge, ols) %>% 
    ggplot() + geom_point(aes(x = y_pred, y = y_obs, colour = procedure), size = 0.5, alpha = 0.2) + 
                scale_color_manual(values = c('red', 'blue')) + 
                stat_smooth(aes(x = y_pred, y = y_obs, colour = procedure), 
                           linetype = 'solid', method = 'lm', se = FALSE, fullrange = TRUE) + 
        geom_abline(slope = 1) + xlab('Predicted y') + ylab('Observed y') +
        theme_bw() + 
    theme(text = element_text(size = 25), plot.title = element_text(hjust = 0.5)) + 
    coord_cartesian(ylim=c(-60, 60))

        
        
ggsave('../writing/ridge_figures/y_obs_vs_pred.png', width = 9, height = 6)

# We explore the distribution of slopes over repeated trials

In [ ]:
n_trials <- 100
# grab a snack this takes a few minutes

lambdas_vs <- rep(0, n_trials)
cv_slope_cs <- rep(0, n_trials)
cv_slope_vs <- rep(0, n_trials)

t0 <- proc.time()
for(i in 1:n_trials){
    data <- draw_cs_vs_sample(x_cs, beta_full, s, sigma, n_vs)  # draw data
    ridge_fit_cv <- cv.glmnet(data$x_cs, data$y_cs, alpha = 0) # get lambda
    min_lambda <- ridge_fit_cv$lambda.min
    
    # predict on training set
    y_pred_cs <- predict(ridge_fit_cv, newx = data$x_cs, s = min_lambda)    
    
    # predict on validation set
    y_pred_vs <- predict(ridge_fit_cv, newx = data$x_vs, s = min_lambda)    
    
    cv_slope_cs[i] <- drop(get_slope(y_pred_cs, data$y_cs))
    cv_slope_vs[i] <- drop(get_slope(y_pred_vs, data$y_vs))
    
    lambdas_vs[i] <- min_lambda
    
}

t1 <- proc.time() - t0
print(t1 - t0)

In [ ]:
data.frame(slopes = c(cv_slope_cs, cv_slope_vs), 
          sample = c(rep('construction', n_trials), rep('validation', n_trials))) %>%
    ggplot() + geom_histogram(aes(x = slopes, fill = sample, color = sample),
                              binwidth = 0.1, position = 'identity', alpha = 0.2) + 
    geom_vline(xintercept = 1)

In [ ]:
# TODO: 
# see how this varies as a function of 1. Sparsity and 2. Signal-to-noise ratio
# now run COPAS shrinkage (maybe just a second CV step) to fix the scaling issue

# Two step lasso

In [ ]:
# we use CV to jointly choose lambda and K
k_range <- seq(0.5, 1.5, by = 0.05)
joint_cv_results <- choose_joint_lambda_k(data$x_cs, data$y_cs, lambdas, k_range, alpha = 0)

In [ ]:
# compare lasso predictions, only choosing lambda vs. k and lambda

# choosing lambda only
ridge_cv_fit <- cv.glmnet(data$x_cs, data$y_cs)
ridge_cv_pred <- predict(ridge_cv_fit, newx = data$x_vs, s = ridge_cv_fit$lambda.min)

# choose both lambda and k
ridge_joint_cv_pred <- joint_cv_results$k * 
    predict(ridge_cv_fit, newx = data$x_vs, s = joint_cv_results$lambda)

In [ ]:
ridge_cv_fit$lambda.min

In [ ]:
print(joint_cv_results$lambda)
print(joint_cv_results$k)

In [ ]:
# compare MSPEs: 
cat('lasso mspe: ', get_mspe(ridge_cv_pred, data$y_vs), '\n')
cat('two-step mspe: ', get_mspe(ridge_joint_cv_pred, data$y_vs))

In [ ]:
# slopes?
cat('lasso slope ', get_slope(ridge_cv_pred, data$y_vs), '\n')
cat('two-step slope: ', get_slope(ridge_joint_cv_pred, data$y_vs))

In [ ]:
data.frame(y_obs = data$y_vs, 
           ridge = as.vector(ridge_cv_pred), 
           two_step = as.vector(ridge_joint_cv_pred)) %>%
    gather(key = 'procedure', value = 'y_pred', ridge, two_step) %>% 
    ggplot() + geom_point(aes(x = y_pred, y = y_obs, colour = procedure), alpha = 0.2) + 
                geom_smooth(aes(x = y_pred, y = y_obs, colour = procedure), 
                           linetype = 'solid', method = 'lm', se = FALSE) + 
        geom_abline(slope = 1)


In [ ]:
# Again run this two-step procedure for a number of trials like before 

In [ ]:
n_trials <- 100
# get another snack ... 

# two rows, n_trials columns
# first row stores the k's, second row stores the lambdas
results_mat <- matrix(0, ncol = n_trials, nrow = 2)
rownames(results_mat) <- c('k', 'lambda')

joint_slope_vs <- rep(0, n_trials)

t0 <- proc.time()

ridge_fit <- glmnet(data$x_cs, data$y_cs, alpha = 0)

for(i in 1:n_trials){
    data <- draw_cs_vs_sample(x_cs, beta_full, s, sigma, n_vs)  # draw data
    
    # choose lambda and k
    joint_results <- choose_joint_lambda_k(data$x_cs, data$y_cs, lambdas, k_range)
    results_mat[1, i] <- joint_results$k
    results_mat[2, i] <- joint_results$lambda
        
    # predict on validation set
    y_pred_vs <- predict(ridge_fit, newx = data$x_vs, s = joint_results$lambda) * joint_results$k 
    ...
    # slope_cs[i] <- drop(get_slope(y_pred_cs, data$y_cs))
    joint_slope_vs[i] <- drop(get_slope(y_pred_vs, data$y_vs))
        
}
t1 <- proc.time() - t0
print(t1 - t0)

In [ ]:
save.image(file = './Apr15_ridge_results.RData')

In [ ]:
data.frame(cv = cv_slope_vs, joint_cv = joint_slope_vs) %>% 
    gather(key = 'method', value = 'slope') %>%
    ggplot() + geom_histogram(aes(x = slope, color = method, fill = method), 
                              position = 'identity', alpha = 0.2, binwidth = 0.1)